## Essential Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
print(num_classes)

2


## Creating a new model

In [ ]:
model = Sequential()
model.add(MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6422656   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 9,651,649
Trainable params: 6,422,785
Non-trainable params: 3,228,864
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
print(test_set)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=1, mode='auto')

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
  # callbacks=[early]
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-Mobilenet-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-Mobilenet-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1648s 5s/step - loss: 2.2111 - accuracy: 0.6728 - mean_absolute_error: 4.2408 - mean_squared_error: 84.8251 - hinge: 2.1960 - binary_crossentropy: 4.2255 - val_loss: 0.4442 - val_accuracy: 0.8814 - val_mean_absolute_error: 1.4714 - val_mean_squared_error: 3.4589 - val_hinge: 0.4329 - val_binary_crossentropy: 1.0017
Epoch 2/20
300/300 [==============================] - 37s 124ms/step - loss: 0.2488 - accuracy: 0.8994 - mean_absolute_error: 1.8277 - mean_squared_error: 5.2697 - hinge: 0.2381 - binary_crossentropy: 0.9377 - val_loss: 0.2890 - val_accuracy: 0.8479 - val_mean_absolute_error: 2.0429 - val_mean_squared_error: 7.2697 - val_hinge: 0.2797 - val_binary_crossentropy: 1.5922
Epoch 3/20
300/300 [==============================] - 37s 123ms/step - loss: 0.1538 - accuracy: 0.9332 - mean_absolute_error: 2.2032 - mean_squared_error: 7.4841 - hinge: 0.1448 - binary_crossentropy: 0.5975 - val_loss: 0.1686 - val_accuracy: 0.9164 - val_me